In [1]:
import cvxpy as cp
import pandas as pd
import numpy as np
import math
import cvxpy_dcopf as cd
import cvxpy_scopf as cs
#import cvxpy_imb as ci
from cvxpylayers.torch import CvxpyLayer
import torch
import torchvision
import timeit
import precontingency as pc
import torch.nn as nn
import time

from tqdm import tqdm
import itertools
import torch.nn.functional as F
from torch import Tensor

from torch.utils.data import DataLoader
import torch.utils.data as Data
from sklearn.model_selection import train_test_split

import psutil
import pickle

import loadsampling as ls

#SOLVER SETTINGS FOR SCS
# used as solver for cvxpylayer in NN
# SCS is standard solver for CVXPYlayer, you can specify other solvers

SCS_solver_args={'use_indirect': False,
         'gpu': False,
         'verbose': False, # False
         'normalize': True, #True heuristic data rescaling
         'max_iters': 10000, #2500 giving the maximum number of iterations
         'scale': 100, #1 if normalized, rescales by this factor
         'eps':1e-3, #1e-3 convergence tolerance
         'cg_rate': 2, #2 for indirect, tolerance goes down like 1/iter^cg_rate
         'alpha': 1.5, #1.5 relaxation parameter
         'rho_x':1e-3, #1e-3 x equality constraint scaling
         'acceleration_lookback': 10, #10
         'write_data_filename':None}


SCS_solver_args2={'use_indirect': False,
         'gpu': False,
         'verbose': False, # False
         'normalize': True, #True heuristic data rescaling
         'max_iters': 20000, #2500 giving the maximum number of iterations
         'scale': 1, #1 if normalized, rescales by this factor
         'eps':1e-3, #1e-3 convergence tolerance
         'cg_rate': 2, #2 for indirect, tolerance goes down like 1/iter^cg_rate
         'alpha': 1.5, #1.5 relaxation parameter
         'rho_x':1e-3, #1e-3 x equality constraint scaling
         'acceleration_lookback': 10, #10
         'write_data_filename':None}


SCS_solver_args3={#'use_indirect': False,
         #'gpu': False,
         'verbose': False, # False
         'normalize': False, #True heuristic data rescaling
         'max_iters': 10000, #2500 giving the maximum number of iterations
         'scale': 1, #1 if normalized, rescales by this factor
         'eps':1e-3, #1e-3 convergence tolerance ????????
         #'cg_rate': 2, #2 for indirect, tolerance goes down like 1/iter^cg_rate ????????
         'alpha': 1.5, #1.5 relaxation parameter
         'rho_x':1e-3, #1e-3 x equality constraint scaling
         'acceleration_lookback': 10, #10
         'write_data_filename':None}

ECOS_solver_args = {"solve_method":"ECOS",
    'verbose': False, # False
    'max_iters': 10000, # Maximum number of iterations
    'reltol': 1e-3, # Convergence tolerance
    # Add other ECOS solver arguments as needed
}


data = pd.read_excel('IEEE27OG.xlsx',sheet_name=None)
Sbase = data['par']['base'][0]
Nloads = len(data['load'])
load_loc = torch.from_numpy(np.array(data['load']['bus']))
Ngens = len(data['gen'])
gen_loc = torch.from_numpy(np.array(data['gen']['bus']))
Nlines = len(data['line'])
Nbus = len(data['bus'])
print(data["line"]['max_f'][0]/Sbase)

max_p = torch.from_numpy(np.array(-data["gen"]['min_p']/Sbase))
min_p = torch.from_numpy(np.array(data["gen"]['max_p']/Sbase))

Nsamples = 1000
Niterations = 100
batch_size = 100
train = 800/1000
test = 200/1000
tollerance_crit = 0.1
learning_rate = 0.01
wd = 0.1 
case = 'N-3'
probabilistic = 'False'

(CVXPY) Nov 15 10:43:15 AM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')
10.0


In [2]:
from torch.nn.parallel import DataParallel

if torch.cuda.is_available():
    # GPU is available
    device = torch.device('cuda')
    print("GPU is available")
else:
    # GPU is not available
    device = torch.device('cpu')
    print("GPU is not available")


if torch.cuda.device_count() >= 1:
    device_ids = list(range(torch.cuda.device_count()))
else:
    device_ids = None
    
num_workers = torch.cuda.device_count()

GPU is not available


In [3]:
#data
#X = np.array(data['load']['p'])/Sbase*np.random.uniform(low=0.75, high=1.25, size = (Nsamples,Nloads))

#LB = 0.75*np.ones(Nloads)
#UB = 1.25*np.ones(Nloads)
#X = np.array(data['load']['p'])/Sbase*ls.kumaraswamymontecarlo(1.6, 2.8, 0.75, LB, UB, Nsamples).T

file_path = 'trained models/training data/Xtrain.pkl'  
with open(file_path, 'rb') as f:
    X = pickle.load(f)

Xtrain = torch.tensor(X[0:int(Nsamples),:],dtype = torch.float32)        
Xtrain_transpose = Xtrain.transpose(0,1)
#Xtest = torch.tensor(X[int(train*Nsamples):,:],dtype = torch.float64)
#standardise
Xmin, Xmax,Xmean,Xstd = np.min(X, axis = 0),np.max(X, axis = 0),np.mean(X, axis = 0),np.std(X, axis = 0)
Xscal = torch.tensor(( X - Xmean ) / Xstd, dtype=torch.float32)
#Xscaltrain = Xscal[0:int(train*Nsamples),:]
#Xscaltest = Xscal[int(train*Nsamples):,:]


c_max = np.max(np.array(data['gen']['cost']))
c_min = np.min(np.array(data['gen']['cost']))

gencost = torch.tensor(np.array(data['gen']['cost']),dtype = torch.float32)  
c_delta = c_max - c_min
cgi0 = torch.tensor((np.array(data['gen']['cost'])-c_min)/c_delta,dtype = torch.float32)
alpha = 1

PTDF = torch.from_numpy(pc.compptdfs(data)).to(torch.float32)

In [4]:
start = timeit.default_timer()
#problem0, Pd0, cg0, cll0, Pgi, Fl0, Th0, lam10, lam20 = cd.create_dcopf_correction(data)
problem0, Pd0, cll0, Pgi, Fl0, Th0, lam10, lam20 = cd.create_dcopf_correction(data)
#problem0, Pd0, Pgi, Fl0, Th0, lam10, lam20 = cd.create_dcopf_correction0(data)
assert problem0.is_dpp()

#cvxpylayer0 = CvxpyLayer(problem0, parameters=[Pd0, cg0, cll0], variables=[Pgi, Fl0, Th0, lam10, lam20])
cvxpylayer0 = CvxpyLayer(problem0, parameters=[Pd0, cll0], variables=[ Pgi, Fl0, Th0, lam10, lam20])
#cvxpylayer0 = CvxpyLayer(problem0, parameters=[Pd0,  Pgi], variables=[Fl0, Th0, lam10, lam20])
time1 = timeit.default_timer() - start
print("Time to create model: ", time1)

max_f = data["line"]['max_f'][0]/Sbase
l = len(data['line'])
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)


Time to create model:  0.14757009999999937
RAM Used (GB): 6.12376576


In [5]:
import pickle
dict_method = 'pickle'

start_load = time.time()
if case == 'N-1':
    if dict_method == 'pickle': 
        file_path_lodf = 'LODF_list/IEEE27/LODF_listN1.pkl'  
        with open(file_path_lodf, 'rb') as f:
            LODF_dict = pickle.load(f)
            
        file_path_SL = 'LODF_list/IEEE27/SL_N1.pkl'
        with open(file_path_SL, 'rb') as f:
            singular_list = pickle.load(f)
    
    Ncontingencies = int(l) 
    tot_lodf_batches = len(LODF_dict)
    lodf_batch_size = Ncontingencies
    last_batch_size = Ncontingencies - (lodf_batch_size*(tot_lodf_batches-1))

if case == 'N-2':
    if dict_method == 'pickle': 
        file_path = 'LODF_list/IEEE27/LODF_listN2.pkl'  
        with open(file_path, 'rb') as f:
            LODF_dict = pickle.load(f)
        
        file_path_SL = 'LODF_list/IEEE27/SL_N2.pkl'
        with open(file_path_SL, 'rb') as f:
            singular_list = pickle.load(f)
        
        print(singular_list)

    Ncontingencies = int(l*(l-1)/2)
    tot_lodf_batches = len(LODF_dict)
    lodf_batch_size = Ncontingencies
    last_batch_size = Ncontingencies - (lodf_batch_size*(tot_lodf_batches-1))
            

if case == 'N-3':        
    if dict_method == 'pickle': 
        file_path = 'LODF_list/IEEE27/LODF_listN3.pkl'  
        with open(file_path, 'rb') as f:
            LODF_dict = pickle.load(f)
        
        file_path_SL = 'LODF_list/IEEE27/SL_N3.pkl'
        with open(file_path_SL, 'rb') as f:
            singular_list = pickle.load(f)
    
    Ncontingencies = int(l*(l-1)/2*(l-2)/3)
    tot_lodf_batches = len(LODF_dict)
    lodf_batch_size = Ncontingencies #20000
    last_batch_size = Ncontingencies - (lodf_batch_size*(tot_lodf_batches-1))
    
if case == 'N-4':
        
    if dict_method == 'pickle': 
        file_path = 'LODF_list/IEEE27/LODF_listN4.pkl'  
        with open(file_path, 'rb') as f:
            LODF_dict = pickle.load(f)
        
        file_path_SL = 'LODF_list/IEEE27/SL_N4.pkl'
        with open(file_path_SL, 'rb') as f:
            singular_list = pickle.load(f)
    
    Ncontingencies = int(l*(l-1)/2*(l-2)/3*(l-3)/4)
    tot_lodf_batches = len(LODF_dict)
    lodf_batch_size = 20000
    last_batch_size = Ncontingencies - (lodf_batch_size*(tot_lodf_batches-1))
    
if case == 'N-5':
        
    if dict_method == 'pickle': 
        file_path = 'LODF_list/IEEE27/LODF_listN5.pkl'  
        with open(file_path, 'rb') as f:
            LODF_dict = pickle.load(f)
        
        file_path_SL = 'LODF_list/IEEE27/SL_N5.pkl'
        with open(file_path_SL, 'rb') as f:
            singular_list = pickle.load(f)
    
    Ncontingencies = int(l*(l-1)/2*(l-2)/3*(l-3)/4*(l-4)/5)
    tot_lodf_batches = len(LODF_dict)
    lodf_batch_size = 20000
    last_batch_size = Ncontingencies - (lodf_batch_size*(tot_lodf_batches-1))
    
if case == 'N-6':
        
    if dict_method == 'pickle': 
        file_path = 'LODF_list/IEEE27/LODF_listN6.pkl'  
        with open(file_path, 'rb') as f:
            LODF_dict = pickle.load(f)
        
        file_path_SL = 'LODF_list/IEEE27/SL_N6.pkl'
        with open(file_path_SL, 'rb') as f:
            singular_list = pickle.load(f)
    
    Ncontingencies = int(l*(l-1)/2*(l-2)/3*(l-3)/4*(l-4)/5*(l-5)/6)
    tot_lodf_batches = len(LODF_dict)
    lodf_batch_size = 20000
    last_batch_size = Ncontingencies - (lodf_batch_size*(tot_lodf_batches-1))
        
    
end_load = time.time()

print('Load LODF time:', end_load - start_load)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)




Load LODF time: 0.004001140594482422
RAM Used (GB): 6.134112256


In [ ]:
l = len(data['line'])

COO_elements = 0
values_tot = 0
lodf_elements = l*l*Ncontingencies
for b in range(len(LODF_dict)):
    lodf_batch = lodf_batch_size
    if b == (tot_lodf_batches - 1): 
        lodf_batch = last_batch_size
                
    index1 = LODF_dict[b][0].detach()
    index2 = LODF_dict[b][1].detach()
    indices = torch.stack((index1,index2)).detach()
    values = LODF_dict[b][2].detach()
    shape = (l,int(l*lodf_batch))
    lodf = torch.sparse_coo_tensor(indices,values,size = shape).coalesce()

    values_coo = lodf.values()
    #print('size value coo tensor:', values_coo.size())
    indices_coo = lodf.indices()
    #print('size index tensor:', indices_coo.size())
    values_tot += torch.numel(values_coo) 
    
    COO_elements += torch.numel(values_coo) 
    
    COO_elements += torch.numel(indices_coo)


print('total elements lodf:', lodf_elements)
element_increase = (COO_elements/lodf_elements)*100 - 100
print('element increase coo:', element_increase, '%')
sparsity = (values_tot/lodf_elements)*100 - 100
print('sparsity:', sparsity, '%')

print('compression:', lodf_elements/COO_elements)

#l = 187
k = 6

sparsity_guarantee = (1-((k*(l-k))/(l**2)))*100
print(sparsity_guarantee)

print(len(singular_list))

In [6]:
#Benchmark either DCOPF or SCOPF
benchmark = 'DCOPF' #'SCOPF'

Pgisscopf = torch.zeros(Ngens,int(Nsamples),dtype = torch.float32)
Fl0sscopf = torch.zeros(Nlines,int(Nsamples),dtype = torch.float32)
Flcs = torch.zeros(Nlines,l,int(Nsamples),dtype = torch.float32)
lam1s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
lam2s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
lam1sc = torch.zeros(Nbus,l,int(Nsamples),dtype = torch.float32)
lam2sc = torch.zeros(Nbus,l,int(Nsamples),dtype = torch.float32)
Th0 = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)

#======== remove islanding cases from contingency list
lcontingencies = range(l)
islanding_cases = [15,20]
lcontingencies_filtered = [l for l in lcontingencies if l not in islanding_cases]
lcontingencies = lcontingencies_filtered

time_scopfs = time.time()

#======== determine infeasible cases
# add loads
load_profile = torch.zeros((Nsamples,Nbus)).to(torch.float32)
load_profile[:, load_loc] = torch.from_numpy(X).to(torch.float32)

if benchmark == 'SCOPF':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf(data, lcontingencies)

elif benchmark == 'SCOPF2':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc = cs.create_scopf2(data, lcontingencies)

elif benchmark == 'DCOPF': 
    #problem0scopf, Pd0scopf, cg0scopf, cl0scopf, Pgiscopf, Fl0scopf, Th0scopf, lam1dc, lam2dc = cd.create_dcopf_upper(data)
    problem0scopf, Pd0scopf, cg0scopf, cll0scopf, Pgiscopf, Fl0scopf, Th0scopf, lam1, lam2 = cd.create_dcopf_lower(data)
    cll0scopf.value = np.zeros(Ngens)

cg0scopf.value = np.array(data['gen']['cost'])
for entry in tqdm(range(int(Nsamples)),position=0, leave=True):   
    Pd0scopf.value = X[entry,:] #np.array(data['load']['p'])/Sbase    
    solution  = problem0scopf.solve(solver=cp.ECOS)    # cp.MOSEK
    if problem0scopf.status in ["infeasible", "unbounded"]:
         # Otherwise, problem.value is inf or -inf, respectively.
         print("Problem infeasible or unbounded")
         #print("Optimal value: %s" % problem0scopf.value)
    Pgisscopf[:,entry] = torch.tensor(Pgiscopf.value)
    Fl0sscopf[:,entry] = torch.tensor(Fl0scopf.value)
    #Flcs[:,:,entry] = insert_zero_columns(torch.tensor(Flc.value),islanding_cases)
    Th0[:,entry] = torch.tensor(Th0scopf.value)
    
    lam1s[:,entry] = torch.tensor(lam1.value)
    lam2s[:,entry] = torch.tensor(lam2.value)
    #lam1sc[:,:,entry] = insert_zero_columns(torch.tensor(lam1c.value),islanding_cases)
    #lam2sc[:,:,entry] = insert_zero_columns(torch.tensor(lam2c.value),islanding_cases)
    
time_scopfs = time.time() - time_scopfs
cost_scopf = torch.matmul(gencost,Pgisscopf)

# add gens
load_profile[:, gen_loc] += Pgisscopf.T

# add smallest values to zero elements
nonzero_values, _ = torch.min(load_profile.masked_fill(load_profile == 0, float('inf')), dim=1, keepdim=True)
mask = load_profile == 0
load_profile = torch.where(mask, nonzero_values, load_profile)


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 328.31it/s]


In [7]:
with torch.set_grad_enabled(True):
    N, D_in, D_out = Nsamples, Nloads, Ngens
    H1, H2, H3 = 8,8,8
   
    #very large network
    model = torch.nn.Sequential(
        torch.nn.Linear(D_in, H1),
        torch.nn.ReLU(),
        nn.Dropout(p=0.2),
        torch.nn.Linear(H1, H2),
        torch.nn.ReLU(),
        nn.Dropout(p=0.2),
        torch.nn.Linear(H2, H3),
        torch.nn.ReLU(),
        nn.Dropout(p=0.2),
        torch.nn.Linear(H3, D_out), #before 23/12 when everything was working!
        #torch.nn.Sigmoid(),
        torch.nn.Tanh(),
        #torch.nn.Softmax(dim=0),
    )
    
    #model = torch.nn.RNN(input_size = D_in, hidden_size = D_out, num_layers=3, nonlinearity = 'tanh', batch_first = True)
    model.double()
    
    def init_weights(u):
        if type(u) == nn.Linear:
            torch.nn.init.xavier_uniform_(u.weight)
            #u.reset_parameters() # default gaussion initializaiton
            torch.nn.init.xavier_uniform_(u.weight, gain=nn.init.calculate_gain('relu'))
            u.bias.data.fill_(0)    
    model.apply(init_weights)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    # adaptive momentum with weight decay
    
    model.to(torch.float32)
    print(model)

print('Load LODF time:', end_load - start_load)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    
num_params_full = sum(p.numel() for p in model.parameters())
#num_params_TT = sum(p.numel() for p in modelTT.parameters())
print('')
print('FC neural network:', num_params_full)
#print('TT neural network:', num_params_TT)
print('input loads:', D_in, 'output gens:', D_out)

Sequential(
  (0): Linear(in_features=21, out_features=8, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=8, out_features=8, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.2, inplace=False)
  (6): Linear(in_features=8, out_features=8, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.2, inplace=False)
  (9): Linear(in_features=8, out_features=10, bias=True)
  (10): Tanh()
)
Load LODF time: 0.004001140594482422
RAM Used (GB): 6.137348096

FC neural network: 410
input loads: 21 output gens: 10


In [8]:
if device_ids:
    model = DataParallel(model, device_ids=device_ids)
    model = model.to(device)
    print(device_ids)

model.to(torch.float32)
print(next(model.parameters()).device)
print(next(model.parameters()).dtype)

# prompt to empty GPU cache: torch.cuda.empty_cache()

cpu
torch.float32


In [9]:
gen_indices = data['gen']['bus'] 
generator_nodes = np.zeros(Nbus)
for index in gen_indices:
    generator_nodes[index] = 1
gen_nodes = np.nonzero(generator_nodes)
gen_nodes = gen_nodes[0].tolist()
print(gen_nodes)
print(gen_indices)

load_indices = data['load']['bus'] 
load_nodes = np.zeros(Nbus)
for index in load_indices:
    load_nodes[index] = 1
load_nodes = np.nonzero(load_nodes)
load_nodes = load_nodes[0].tolist()
print(load_nodes)

[1, 5, 9, 16, 18, 19, 21, 25, 26]
0     1
1     9
2    16
3    16
4    18
5    19
6    21
7    25
8    26
9     5
Name: bus, dtype: int64
[0, 2, 3, 5, 6, 7, 8, 10, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26]


In [ ]:
start = time.time()
max_p = torch.from_numpy(np.array(-data["gen"]['min_p']/Sbase))
min_p = torch.from_numpy(np.array(data["gen"]['max_p']/Sbase))

#============== Initialize training testing data
Xscal_train, Xscal_test, cost_scopf_train, cost_scopf_test = train_test_split(Xscal, cost_scopf, test_size=1-train, random_state=42)

training_dataset = Data.TensorDataset(Xscal_train, cost_scopf_train)
testing_dataset = Data.TensorDataset(Xscal_test, cost_scopf_test)

train_loader = Data.DataLoader(dataset=training_dataset, batch_size=batch_size, shuffle=True)
test_loader = Data.DataLoader(dataset=testing_dataset, batch_size=batch_size, shuffle=False)  # Note: shuffle=False for testing

import torch.nn.functional as F

flow_penalty = 0.1
crit_penalty = 0
infeasibility_penalty = 0
cost_penalty = 5
soft_ineq_penalty = 100
soft_eq_penalty = 2000

#================ Initialize tensors
error = np.zeros(shape=(Niterations,2))
avg_infeasibility = np.zeros(shape=(Niterations,2))
relcost = np.zeros(shape=(Niterations,2))

time_CF = np.zeros(shape=(Niterations,1))
memory_CF = np.zeros(shape=(Niterations,1))
percentage_CF = np.zeros(shape=(Niterations,1))
backward_CF = np.zeros(shape=(Niterations,1))

Pgis = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
Flis = torch.zeros(size = (Nlines,int(batch_size)),dtype = torch.float32)
clil = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
infeasibility = torch.zeros(int(batch_size), requires_grad = False)

relcotrain_epoch = torch.zeros(size = (int(Nsamples*(train)/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)
relcotest_epoch = torch.zeros(size = (int(Nsamples*(test)/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)

imbalancetrain_epoch = torch.zeros(size = (int(Nsamples*(train)/batch_size),1),dtype = torch.float32, requires_grad = False)
imbalancetest_epoch = torch.zeros(size = (int(Nsamples*(test)/batch_size),1),dtype = torch.float32, requires_grad = False)

infeasibilitytrain_epoch = torch.zeros(size = (int(Nsamples*(train)/batch_size),Nbus),dtype = torch.float32, requires_grad = False)
infeasibilitytest_epoch = torch.zeros(size = (int(Nsamples*(test)/batch_size),Nbus),dtype = torch.float32, requires_grad = False)
infeasibility_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

line_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
crit_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
avg_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
count_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

violation_percentage = torch.zeros(size = (int(Nsamples*train/batch_size),1),dtype = torch.float32, requires_grad = False)
time_contflow = torch.zeros(size = (int(Nsamples*train/batch_size),1),dtype = torch.float32, requires_grad = False)
memory_contflow = torch.zeros(size = (int(Nsamples*train/batch_size),1),dtype = torch.float32, requires_grad = False)
backward_contflow = torch.zeros(size = (int(Nsamples*train/batch_size),1),dtype = torch.float32, requires_grad = False)

print("It |   Time   |    Imbalance    | Mod out  |   Infeasibility   |  Rel cost")
for epoch in range(Niterations):
    
    time_nnSCS = time.time()
    for i, (Xbatch, cost_scopf_batch) in enumerate(tqdm(train_loader)):
            start_forward = time.time()
            start_forward_memory = psutil.virtual_memory()[3] / 1000000000            
            
            #============= initialize
            model.train()
            optimizer.zero_grad() 
            model.zero_grad()
            Xbatch = Xbatch.clone().detach().to(device)
            cost_scopf_batch = cost_scopf_batch.clone().detach().to(device)

            Xtrain_batch = ((Xbatch.cpu() * Xstd) + Xmean).to(torch.float32).to(device)
            load_profile = torch.zeros((batch_size, Nbus)).to(device)
            #load_profile[:, load_loc] = Xtrain_batch

            #================== Perform forward pass
            start_FP = time.time()
            clil = (model(Xbatch) + 1) / 2
            end_FP = time.time()
            
            #============= soft violation base case
            #===================== line flow limits
            Pg_guess = torch.mul(max_p,clil) 
            columns_to_sum = [3,4]
            sum_result = torch.sum(Pg_guess[:, columns_to_sum], dim=1).unsqueeze(1)
            modified_tensor = torch.cat((Pg_guess[:, :3], sum_result, Pg_guess[:, 5:]), dim=1)
            Pg_mod = modified_tensor
            
            Pg_train = torch.zeros((batch_size,Nbus)).to(torch.float32)
            Pd_train = torch.zeros((batch_size,Nbus)).to(torch.float32)

            Pg_train[:,gen_nodes] = Pg_mod.to(torch.float32)
            Pd_train[:,load_nodes] = Xtrain_batch.to(torch.float32)
                
            Fl_guess = PTDF@(Pg_train-Pd_train).T
            Fl_base = F.relu(torch.abs(Fl_guess)-max_f).sum()/int(batch_size)
            
            #================= power balance
            Pg_tot = Pg_guess.sum(dim=1)
            Pd_tot = Xtrain_batch.sum(dim=1)
            eq_imbalance = torch.abs(Pg_tot - Pd_tot).sum()/int(batch_size)
            eq_infeasible = torch.abs(Pg_tot - Pd_tot)
            eq_infeasible[torch.abs(eq_infeasible) < 0.1*Pg_tot] = 0
            
            #============= cvxpylayer
            start_cvxpy = time.time()
            Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtrain_batch, clil, solver_args=ECOS_solver_args)
            Pgis = Pgi.T # 9x100
            Flis = Fli.T.to(torch.float32) #33x100
            end_cvxpy = time.time()
            
            #=============== Without correction layer
            Pgis = Pg_guess.T.to(torch.float32)
            Flis = Fl_guess.to(torch.float32)
                       
            #================ infeasibility
            infeasibility_tensor = (lam1i + lam2i)
            infeasibility_loss = infeasibility_tensor.sum() / int(batch_size)
            infeasibility_batch = infeasibility_tensor.sum(dim=0).detach()

            infeasibility_tensor[abs(infeasibility_tensor) < 1e-5] = 0
            infeasible_buses = (abs(infeasibility_tensor) > 1e-5)

            infeasible_cases = torch.zeros((batch_size, 1)).to(device)
            row_has_true = torch.any(infeasible_buses, axis=1)
            infeasible_cases[row_has_true] = True
            infeasible_cases = infeasible_cases.sum()
            
            #================== without correction layer
            infeasible_cases = torch.zeros((batch_size, 1)).to(device)
            row_has_true_ineq = torch.any(F.relu(torch.abs(Fl_guess)-max_f), axis=0)
            row_has_true_eq = torch.any(eq_infeasible.unsqueeze(1), axis = 1)
            infeasible_cases[row_has_true_ineq] = True
            infeasible_cases[row_has_true_eq] = True
            infeasible_cases = infeasible_cases.sum()

            #==================== cost
            gencost = gencost.requires_grad_()
            
            batch_cost = torch.matmul(gencost, Pgis)
            batch_relco = ((batch_cost - cost_scopf_batch) / cost_scopf_batch) * 100
            relco_avg = torch.mean(batch_relco)

            cost_loss = torch.mean(batch_cost) #/ int(batch_size)

            #===================== compute contingency flows for each sample  
            imbalancetrain_batch = 0
            critical_loss = 0
            count_crit_violation = 0
            count_line_violation = 0
            count_violation = 0

            for b in range(tot_lodf_batches):
                lodf_batch = lodf_batch_size
                if b == (tot_lodf_batches - 1): 
                    lodf_batch = last_batch_size
                left_ones = torch.ones((lodf_batch, 1)).to(device)
                right_ones = torch.ones((int(l * batch_size), 1)).to(device)
                Fli_0 = Flis.expand(lodf_batch, -1, -1)

                index1 = LODF_dict[b][0].detach()
                index2 = LODF_dict[b][1].detach()
                indices = torch.stack((index1, index2)).detach()
                values = LODF_dict[b][2].detach()
                shape = (l, int(l * lodf_batch))
                lodf = torch.sparse_coo_tensor(indices, values, size=shape, requires_grad=False).T.to(device).detach()

                imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, batch_size))) - max_f)).detach().to(device)
                count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > tollerance_crit * max_f, dim=1)
                count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

                mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
                indices_remove = mask.nonzero().squeeze(1)
                num_zero_rows = torch.sum(mask)
                batch_percentage_violation = (num_zero_rows.item() / len(imbalancetrain_batch_loop[:, 0])) * 100

                mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
                mask_flow[indices_remove] = False
                lodf_test = lodf.to_dense()[mask_flow]
                lodf_test = lodf_test.clone().detach()
                lodf = lodf_test.to_sparse_coo().requires_grad_().to(device)


                Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow].to(device)
                imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis)) - max_f)).to(device)
                critical_violation_penalty = imbalancetrain_batch_loop[imbalancetrain_batch_loop > tollerance_crit * max_f]
                count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)  

                imbalancetrain_batch += torch.sum(imbalancetrain_batch_loop) / int(batch_size)
                critical_loss += torch.sum(critical_violation_penalty)/int(batch_size)
                
                count_violation += count_violation_loop
                count_crit_violation += torch.count_nonzero(count_crit_violation_loop.detach())
                count_line_violation += torch.count_nonzero(count_line_violation_loop.detach())

            end_forward = time.time()
            end_forward_memory = psutil.virtual_memory()[3] / 1000000000
            
            memory_contflow_batch = end_forward_memory - start_forward_memory
            time_contflow_batch = end_forward - start_forward
            
            #======================== Perform back pass
            loss = flow_penalty * imbalancetrain_batch + infeasibility_penalty * infeasibility_loss \
            + cost_penalty * cost_loss + Fl_base * soft_ineq_penalty + eq_imbalance * soft_eq_penalty

            start_BP = time.time()
            loss.backward()
            end_BP = time.time()
            backward_batch = end_BP - start_BP
            #print('Backward pass:', end_BP - start_BP)

            optimizer.step()

            relcotrain_epoch[i,:] = batch_relco.detach()
            imbalancetrain_epoch[i,:] = imbalancetrain_batch.detach()
            infeasibilitytrain_epoch[i,:] = infeasibility_batch.detach()
            infeasibility_count[i,:] = infeasible_cases.detach()
            line_violation_count[i,:] = count_line_violation.detach()
            crit_violation_count[i,:] = count_crit_violation.detach()
            #avg_violation_list[i,:] = avg_violation.detach()
            count_violation_list[i,:] = count_violation
            
            violation_percentage[i,:] = batch_percentage_violation
            time_contflow[i,:] = time_contflow_batch
            memory_contflow[i,:] = memory_contflow_batch
            backward_contflow[i,:] = backward_batch
    
    with torch.no_grad():
        for i, (Xbatch, cost_scopf_batch) in enumerate(tqdm(test_loader)):
                
                model.eval() # deactivate dropout

                Xbatch = Xbatch.clone().detach().to(device)
                cost_scopf_batch = cost_scopf_batch.clone().detach().to(device)

                Xtest_batch = ((Xbatch.cpu() * Xstd) + Xmean).to(torch.float32).to(device)

                #================== Perform forward pass
                clil = (model(Xbatch) + 1) / 2
                
                #============= soft violation base case
                Pg_guess = torch.mul(max_p,clil) 
                columns_to_sum = [3,4]
                sum_result = torch.sum(Pg_guess[:, columns_to_sum], dim=1).unsqueeze(1)
                modified_tensor = torch.cat((Pg_guess[:, :3], sum_result, Pg_guess[:, 5:]), dim=1)
                Pg_mod = modified_tensor

                Pg_train = torch.zeros((batch_size,Nbus)).to(torch.float32)
                Pd_train = torch.zeros((batch_size,Nbus)).to(torch.float32)

                Pg_train[:,gen_nodes] = Pg_mod.to(torch.float32)
                Pd_train[:,load_nodes] = Xtrain_batch.to(torch.float32)

                Fl_guess = PTDF@(Pg_train-Pd_train).T
                #Fl_base = F.relu(torch.abs(Fl_guess)-max_f).sum()/int(batch_size)

                #============= cvxpylayer
                Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtest_batch, clil, solver_args=ECOS_solver_args)
                Pgis = Pgi.T
                Flis = Fli.T.to(torch.float32)
                
                #=============== Without correction layer
                Pgis = Pg_guess.T.to(torch.float32)
                Flis = Fl_guess.to(torch.float32)

                #================ compute cost and infeasibility
                infeasibility_batch = torch.sum(lam1i) + torch.sum(lam2i)

                batch_cost = torch.matmul(gencost, Pgis)
                batch_relco = ((batch_cost - cost_scopf_batch) / cost_scopf_batch) * 100

                #===================== compute contingency flows for each sample
                imbalancetest_batch = 0

                for b in range(tot_lodf_batches):
                    lodf_batch = lodf_batch_size
                    if b == (tot_lodf_batches - 1): 
                        lodf_batch = last_batch_size
                    left_ones = torch.ones((lodf_batch, 1)).to(device)
                    right_ones = torch.ones((int(l * batch_size), 1)).to(device)
                    Fli_0 = Flis.expand(lodf_batch, -1, -1).detach()

                    index1 = LODF_dict[b][0].detach()
                    index2 = LODF_dict[b][1].detach()
                    indices = torch.stack((index1, index2)).detach()
                    values = LODF_dict[b][2].detach()
                    shape = (l, int(l * lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices, values, size=shape).T.to(device)

                    imbalancetest_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, batch_size))) - max_f))
                    imbalancetest_batch += torch.sum(imbalancetest_batch_loop) / int(batch_size)

        
                relcotest_epoch[i,:] = batch_relco.detach()
                imbalancetest_epoch[i,:] = imbalancetest_batch.detach()
                infeasibilitytest_epoch[i,:] = infeasibility_batch.detach()
            
    time_nnSCS = time.time() - time_nnSCS  

    imbalancetrain = torch.mean(imbalancetrain_epoch).detach()
    imbalancetest = torch.mean(imbalancetest_epoch).detach()
    
    infeasibilitytrain = torch.mean(infeasibilitytrain_epoch).detach()
    infeasibilitytest = torch.mean(infeasibilitytest_epoch).detach()
    
    relcotrain = torch.mean(relcotrain_epoch).detach()
    relcotest = torch.mean(relcotest_epoch).detach()
    
    #====== convergence criteria
    infeasible_cases_tot = torch.sum(infeasibility_count)
    line_violation_tot = torch.sum(line_violation_count)
    crit_violation_tot = torch.sum(crit_violation_count)
    #tot_avg_violation = torch.sum(avg_violation_list).detach()
    count_violation_tot = torch.sum(count_violation_list).detach()
    
    tot_violation_percentage = torch.mean(violation_percentage)
    tot_time_contflow = torch.sum(time_contflow)
    tot_memory_contflow = torch.mean(memory_contflow)
    tot_backward_contflow = torch.sum(backward_contflow)
    
    #============= 
    error[epoch,:] = torch.stack((imbalancetrain,imbalancetest)).detach().numpy()
    avg_infeasibility[epoch,:] =  torch.stack((infeasibilitytrain,infeasibilitytest)).detach().numpy()
    relcost[epoch,:] =  torch.stack((relcotrain,relcotest)).detach().numpy()
    
    #================= computational graph
    time_CF[epoch,:] = tot_time_contflow
    memory_CF[epoch,:] = tot_memory_contflow 
    percentage_CF[epoch,:] = tot_violation_percentage
    backward_CF[epoch,:] = tot_backward_contflow

    print(epoch, " | ", round(time_nnSCS,0), " | ",np.round(error[epoch,:],3), " | ", np.round(torch.mean(clil[:]).cpu().detach().numpy(),3), " | ", np.round(avg_infeasibility[epoch,:],3), " | ", np.round(relcost[epoch,:], 3) )
    
    #======================= Convergence check infeasible cases
    print('current infeasibility:', (infeasible_cases_tot/int(train*Nsamples))*100, '%', 'goal:', 1, '%')
    
    print('infeasible contingency cases:', line_violation_tot, (line_violation_tot/(Ncontingencies*Nsamples))*100, '%')
    print('10%+ contingency cases:', crit_violation_tot, (crit_violation_tot/(Ncontingencies*Nsamples))*100, '%')
    print('number of violations:', count_violation_tot)
    print('percentage lines not in violation:', tot_violation_percentage, '%')
    
    print('flow penalty:', flow_penalty*imbalancetrain_batch)
    print('infeasible penalty:', infeasibility_penalty*infeasibility_loss)
    print('cost penalty:', cost_penalty*cost_loss)
    print('soft ineq loss:', soft_ineq_penalty*Fl_base)
    print('soft eq loss:', soft_eq_penalty*eq_imbalance)
    #print('extra;', loss_term)
    
        
end = time.time()
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)



In [ ]:
if case == 'N-1':
    file_path = 'trained models/correction/without correct/ieee39_N1.pt'
if case == 'N-2':
    file_path = 'trained models/correction/without correct/ieee39_N2.pt'
if case == 'N-3':
    file_path = 'trained models/correction/without correct/ieee39_N3.pt'
if case == 'N-4':
    file_path = 'trained models/correction/without correct/ieee39_N4.pt'
if case == 'N-5':
    file_path = 'trained models/correction/without correct/ieee39_N5.pt'

# Save the model
#torch.save(model.state_dict(), file_path)

In [10]:
if case == 'N-1':
    file_path = 'trained models/correction/ieee39_N1_MC_200.pt'
if case == 'N-2':
    file_path = 'trained models/correction/ieee39_N2_MC_500.pt'
if case == 'N-3':
    file_path = 'trained models/correction/ieee39_N3_MC_500.pt'
if case == 'N-4':
    file_path = 'trained models/correction/ieee39_N4_MC_500.pt'
if case == 'N-5':
    file_path = 'trained models/correction/ieee39_N5_MC_500.pt'

# Load the saved model state dictionary
model.load_state_dict(torch.load(file_path))

# Set the model to evaluation mode
model.eval()
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

RAM Used (GB): 6.10349056


In [ ]:
#======== get values for computational graph
comp_graph = np.stack((percentage_CF, time_CF, memory_CF, backward_CF))

if case == 'N-1':
    file_path = 'Computational graph data/IEEE39/reduced graph/Comp_graphN1.pkl'
if case == 'N-2':
    file_path = 'Computational graph data/IEEE39/reduced graph/Comp_graphN2.pkl'
if case == 'N-3':
    file_path = 'Computational graph data/IEEE39/reduced graph/Comp_graphN3.pkl'
if case == 'N-4':
    file_path = 'Computational graph data/IEEE39/reduced graph/Comp_graphN4.pkl'
if case == 'N-5':
    file_path = 'Computational graph data/IEEE39/reduced graph/Comp_graphN5.pkl'

# Save the dictionary to the specified file path using pickle
with open(file_path, 'wb') as f:
    pickle.dump(comp_graph, f)

In [ ]:
totaltime = start - end
print("total time:", totaltime)

import matplotlib.pyplot as plt
plt.plot(error[:l,0], 'r', label='Training error')
plt.plot(error[:l,1], 'b', label='Testing error')
#new_list = 10*range(0, 10,l+1)
#plt.xticks(new_list)
plt.ylabel('Errors')
plt.xlabel('Iterations')
plt.legend()
plt.show()
plt.close()

In [11]:
Nsamples = 1000
#X = np.array(data['load']['p'])/Sbase*ls.kumaraswamymontecarlo(1.6, 2.8, 0.75, LB, UB, Nsamples).T
file_path = 'trained models/training data/Xtest1.pkl' 
with open(file_path, 'rb') as f:
    X = pickle.load(f)
Xtrain = torch.tensor(X[0:int(Nsamples),:],dtype = torch.float32)        
Xtrain_transpose = Xtrain.transpose(0,1)
#Xtest = torch.tensor(X[int(train*Nsamples):,:],dtype = torch.float64)
#standardise
Xmin, Xmax,Xmean,Xstd = np.min(X, axis = 0),np.max(X, axis = 0),np.mean(X, axis = 0),np.std(X, axis = 0)
Xscal = torch.tensor(( X - Xmean ) / Xstd, dtype=torch.float32)

In [12]:
#=============== Benchmark #1 contingency screening N-k
benchmark = 'SCOPF3screening' #'SCOPF'

if benchmark == 'SCOPF':
    lcontingencies = range(l)
    islanding_cases = [15,20]
    lcontingencies_filtered = [l for l in lcontingencies if l not in islanding_cases]
    lcontingencies = lcontingencies_filtered
    Nk_contingencies = lcontingencies
elif benchmark == 'SCOPF1screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),1)))
elif benchmark == 'SCOPF2screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),2)))
elif benchmark == 'SCOPF3screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),3)))
elif benchmark == 'SCOPF4screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),4)))
elif benchmark == 'SCOPF5screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),5)))

Pgisscopf = torch.zeros(Ngens,int(Nsamples),dtype = torch.float32)
Fl0sscopf = torch.zeros(Nlines,int(Nsamples),dtype = torch.float32)
#Flcs = torch.zeros(Nlines,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
lam1s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
lam2s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
#lam1sc = torch.zeros(Nbus,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
#lam2sc = torch.zeros(Nbus,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
Th0 = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)

#======== remove islanding cases from contingency list
if benchmark != 'SCOPF':
    lcontingencies = [0]
screening_iterations = 4

time_scopfs = time.time()

for i in range(screening_iterations):
    imbalance_list = []
    Nk_zeros = list(range(len(Nk_contingencies)))
    zero_indices = [l for l in Nk_zeros if l not in lcontingencies]
    print(lcontingencies)

    if benchmark == 'SCOPF':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf(data, lcontingencies)
    elif benchmark == 'SCOPF1screening':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf1_screening(data, lcontingencies)
    elif benchmark == 'SCOPF2screening':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf2_screening(data, lcontingencies)
    elif benchmark == 'SCOPF3screening':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf3_screening(data, lcontingencies)
    elif benchmark == 'SCOPF4screening':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf4_screening(data, lcontingencies)
    elif benchmark == 'SCOPF5screening':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf5_screening(data, lcontingencies)
   

    cg0scopf.value = np.array(data['gen']['cost'])
    for entry in tqdm(range(int(Nsamples)),position=0, leave=True):   
        Pd0scopf.value = X[entry,:] #np.array(data['load']['p'])/Sbase    
        try:
            solution  = problem0scopf.solve(solver=cp.ECOS)    # cp.MOSEK
            if problem0scopf.status in ["infeasible", "unbounded"]:
                # Otherwise, problem.value is inf or -inf, respectively.
                print("Problem infeasible or unbounded")
                #print("Optimal value: %s" % problem0scopf.value)
            Pgisscopf[:,entry] = torch.tensor(Pgiscopf.value)
            Fl0sscopf[:,entry] = torch.tensor(Fl0scopf.value)
            Th0[:,entry] = torch.tensor(Th0scopf.value)

            lam1s[:,entry] = torch.tensor(lam1.value)
            lam2s[:,entry] = torch.tensor(lam2.value)
        except cp.error.SolverError:
            Pgisscopf[:,entry] = Pgisscopf[:,(entry-1)]
            Fl0sscopf[:,entry] = Fl0sscopf[:,(entry-1)]
            Th0[:,entry] = Th0[:,(entry-1)]

            lam1s[:,entry] = lam1s[:,(entry-1)]
            lam2s[:,entry] = lam2s[:,(entry-1)]
            
            if i == (screening_iterations - 1):
                Xtrain[entry,:] = Xtrain[(entry-1),:]
            
            print("Solver unable to solve for entry:", entry)
            continue
        
    
    cost_scopf = torch.matmul(gencost,Pgisscopf)
    
    if benchmark == 'SCOPF':
        break

    #============ compute Nk flows
    Flis = Fl0sscopf
                
    for b in (range(tot_lodf_batches)):

        lodf_batch = lodf_batch_size
        if b == (tot_lodf_batches - 1):
            lodf_batch = last_batch_size
        left_ones = torch.ones((lodf_batch,1))
        right_ones = torch.ones((int(l*batch_size),1))
        Fli_0 = Flis.expand(lodf_batch, -1, -1)

        summed_tensor = torch.zeros(lodf_batch).to(device)

        index1 = LODF_dict[b][0].detach()
        index2 = LODF_dict[b][1].detach()
        indices = torch.stack((index1,index2)).detach()
        values = LODF_dict[b][2].detach()
        shape = (l,int(l*lodf_batch))
        lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

        for k in range(int(Nsamples/batch_size)):
            start = k*batch_size
            end = (k+1)*batch_size
            imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0[:,:,start:end] + torch.sparse.mm(lodf,Flis[:,start:end]).reshape((lodf_batch,l,batch_size)))- max_f)).detach()

            #======= get indices of contingencies of highest violations
            summed_tensor += torch.sum(torch.sum(imbalancetrain_batch_loop, dim=1), dim=1)

        if b == 0:
            desired_length = len(summed_tensor)

        padding_length = desired_length - len(summed_tensor)
        padded_tensor = F.pad(summed_tensor, (0, padding_length), value=0)
        imbalance_list.extend(padded_tensor)


    indices_per_iteration = 20 # len(summed_tensor)

    stacked_imbalance = torch.stack(imbalance_list, dim=0)
    highest_indices = (np.argsort(stacked_imbalance.cpu())[-indices_per_iteration:]).tolist()
    new_indices = [idx for idx in highest_indices if idx not in lcontingencies] # Filter out the indices that are already in the list
    lcontingencies.extend(new_indices) # Append the new indices to the list

    
time_scopfs = time.time() - time_scopfs
print('contingency screening time:', time_scopfs)

[0]


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 187.63it/s]


[0, 3478, 199, 3185, 2586, 3162, 2563, 3262, 3244, 3440, 2608, 3187, 3156, 2557, 2580, 662, 1904, 3437, 197, 2859, 2857]


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:44<00:00, 22.59it/s]


[0, 3478, 199, 3185, 2586, 3162, 2563, 3262, 3244, 3440, 2608, 3187, 3156, 2557, 2580, 662, 1904, 3437, 197, 2859, 2857, 35, 39, 53, 55, 40, 36, 32, 37, 38, 45, 47, 33, 1066, 1271, 1295, 1054, 1067, 1272, 51, 46]


 25%|███████████████████▊                                                           | 251/1000 [00:24<00:56, 13.22it/s]C:\Users\bgiraud\AppData\Local\anaconda3\envs\learnSCOPF\lib\site-packages\cvxpy\problems\problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:23<00:00, 11.99it/s]


[0, 3478, 199, 3185, 2586, 3162, 2563, 3262, 3244, 3440, 2608, 3187, 3156, 2557, 2580, 662, 1904, 3437, 197, 2859, 2857, 35, 39, 53, 55, 40, 36, 32, 37, 38, 45, 47, 33, 1066, 1271, 1295, 1054, 1067, 1272, 51, 46, 511, 1858, 1086, 395, 2911, 2866, 2936]


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:42<00:00,  9.77it/s]


contingency screening time: 245.28280377388


In [13]:
##============= check infeasibility
infeasibility_base = (lam1s + lam2s).permute(1,0)
infeasibility_base[abs(infeasibility_base) < 1e-4] = 0
infeasible_buses_base = (abs(infeasibility_base) > 1e-4) #0.001*load_profile) # 10% of total load available for generation/load shedding
scopf_cost_average = cost_scopf.mean()

infeasible_cases_base = torch.zeros((Nsamples,1))
row_has_true_base = torch.any(infeasible_buses_base, axis=1)
infeasible_cases_base[row_has_true_base] = True
infeasible_cases_tot_base = infeasible_cases_base.sum()
print('infeasible base cases:', infeasible_cases_tot_base)
print('percentage base infeasibility:', (infeasible_cases_tot_base/(Nsamples))*100, '%')
print('total average cost:', scopf_cost_average)

print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

infeasible base cases: tensor(0.)
percentage base infeasibility: tensor(0.) %
total average cost: tensor(2144.4783)
RAM Used (GB): 6.25453056


In [14]:
#=========== check how many violations N-k
Flis = Fl0sscopf
    
count_crit_violation = 0
count_line_violation = 0

for b in (range(tot_lodf_batches)):
    lodf_batch = lodf_batch_size
    if b == (tot_lodf_batches - 1): 
        lodf_batch = last_batch_size
    left_ones = torch.ones((lodf_batch,1))
    right_ones = torch.ones((int(l*batch_size),1))
    Fli_0 = Flis.expand(lodf_batch, -1, -1)

    index1 = LODF_dict[b][0].detach()
    index2 = LODF_dict[b][1].detach()
    indices = torch.stack((index1,index2)).detach()
    values = LODF_dict[b][2].detach()
    shape = (l,int(l*lodf_batch))
    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

    # lodf[(l*Ncont),l] x flis[l,batch_size]
    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf,Flis).reshape((lodf_batch,l,Nsamples)))- max_f)).detach()
    count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > tollerance_crit*max_f, dim=1)
    count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
    violation_indices_scopf = torch.nonzero(count_line_violation_loop, as_tuple=False)
    

    count_crit_violation += torch.count_nonzero(count_crit_violation_loop)
    count_line_violation += torch.count_nonzero(count_line_violation_loop)

crit_violation_tot = count_crit_violation
line_violation_tot = count_line_violation

print('infeasible N-k contingency cases:', line_violation_tot, (line_violation_tot/(Ncontingencies*Nsamples))*100, '%')
print('10%+ contingency cases:', crit_violation_tot, (crit_violation_tot/(Ncontingencies*Nsamples))*100, '%')


infeasible N-k contingency cases: tensor(179971) tensor(3.2986) %
10%+ contingency cases: tensor(43209) tensor(0.7920) %


In [15]:
start_test = time.time()
#============== Initialize training testing data
testing_dataset = Data.TensorDataset(Xscal, cost_scopf)
test_loader = Data.DataLoader(dataset=testing_dataset, batch_size=batch_size, shuffle=False)  # Note: shuffle=False for testing
import torch.nn.functional as F

#================ Initialize tensors
error = np.zeros(shape=(Niterations,1))
avg_infeasibility = np.zeros(shape=(Niterations,1))
relcost = np.zeros(shape=(Niterations,1))
cost = np.zeros(shape=(Niterations,1))

Pgis = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
Flis = torch.zeros(size = (Nlines,int(batch_size)),dtype = torch.float32)
clil = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
infeasibility = torch.zeros(int(batch_size), requires_grad = False)

relcotest_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)
cost_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)

imbalancetest_epoch = torch.zeros(size = (round((Nsamples)/batch_size),1),dtype = torch.float32, requires_grad = False)

infeasibilitytest_epoch = torch.zeros(size = (int(Nsamples/batch_size),Nbus),dtype = torch.float32, requires_grad = False)
infeasibility_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

line_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
crit_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
avg_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
count_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

print("It |   Time   |    Imbalance    | Mod out  |   Infeasibility   |  Rel cost")
for epoch in range(1):    
    with torch.no_grad():
        
        for i, (Xbatch, cost_scopf_batch) in enumerate(tqdm(test_loader)):
                
                model.eval() # deactivate dropout, when I changed this, testing error stayed same

                Xbatch = Xbatch.clone().detach()
                cost_scopf_batch = cost_scopf_batch.clone().detach()

                Xtest_batch = ((Xbatch * Xstd)+Xmean).to(torch.float32)

                #================== Perform forward pass
                #clil = (model(Xbatch))
                clil = (model(Xbatch)+1)/2
                
                #============= cvxpylayer                
                Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtest_batch, clil, solver_args=ECOS_solver_args)
                Pgis = Pgi.T
                Flis = Fli.T.to(torch.float32)
                
                #================ compute cost and infeasibility
                infeasibility_batch = sum(lam1i.detach().numpy()) + sum(lam2i.detach().numpy())
                infeasibility_batch = torch.tensor(infeasibility_batch)
                infeasibility_tensor = lam1i + lam2i
                infeasibility_tensor[abs(infeasibility_tensor) < 1e-5 ] = 0
                infeasible_buses = (abs(infeasibility_tensor) > 1e-5)
                
                infeasible_cases = torch.zeros((batch_size,1))
                row_has_true = torch.any(infeasible_buses, axis=1)
                infeasible_cases[row_has_true] = True
                infeasible_cases = infeasible_cases.sum()
                               
                batch_cost = torch.matmul(gencost,Pgis)
                batch_relco = ((batch_cost-cost_scopf_batch)/cost_scopf_batch)*100

                #===================== compute contingency flows for each sample
                imbalancetest_batch = 0
                count_crit_violation = 0
                count_line_violation = 0
                count_violation = 0

                for b in (range(tot_lodf_batches)):
                    lodf_batch = lodf_batch_size
                    if b == (tot_lodf_batches - 1): 
                        lodf_batch = last_batch_size
                    left_ones = torch.ones((lodf_batch_size,1))
                    right_ones = torch.ones((int(l*batch_size),1))
                    Fli_0 = Flis.expand(lodf_batch, -1, -1)#.contiguous().view(-1, len(Flis[1])).detach()#.to(torch.float32).clone()

                    index1 = LODF_dict[b][0].detach()
                    index2 = LODF_dict[b][1].detach()
                    indices = torch.stack((index1,index2)).detach()
                    values = LODF_dict[b][2].detach()
                    shape = (l,int(l*lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices,values,size = shape).T.to(device)

                    imbalancetest_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf,Flis).reshape((lodf_batch,l,batch_size)))- max_f))
                    imbalancetest_batch_loop[abs(imbalancetest_batch_loop) < 1e-5] = 0
                    
                    count_crit_violation_loop = torch.sum(imbalancetest_batch_loop > tollerance_crit*max_f, dim=1)
                    count_line_violation_loop = torch.sum(imbalancetest_batch_loop > 0, dim=1)
                    batch_nonzero_indices = torch.nonzero(count_line_violation_loop, as_tuple=False)
                    batch_nonzero_indices[:, 1] += i*100
                    violation_indices_nn = torch.cat((violation_indices_nn, batch_nonzero_indices), dim=0)
                    
                    imbalancetest_batch_loop = imbalancetest_batch_loop.reshape(lodf_batch,int(l*batch_size))
                    count_violation_loop = torch.count_nonzero(imbalancetest_batch_loop)
                
                    imbalancetest_batch += torch.sum(imbalancetest_batch_loop)/int(batch_size)
                
                    count_violation += count_violation_loop
                    count_crit_violation += torch.count_nonzero(count_crit_violation_loop.detach())
                    count_line_violation += torch.count_nonzero(count_line_violation_loop.detach())
        
                relcotest_epoch[i,:] = batch_relco.detach()
                cost_epoch[i,:] = batch_cost.detach()
                imbalancetest_epoch[i,:] = imbalancetest_batch.detach()
                infeasibilitytest_epoch[i,:] = infeasibility_batch.detach()
                
                infeasibility_count[i,:] = infeasible_cases.detach()
                line_violation_count[i,:] = count_line_violation.detach()
                crit_violation_count[i,:] = count_crit_violation.detach()
                #avg_violation_list[i,:] = avg_violation.detach()
                count_violation_list[i,:] = count_violation
             

    imbalancetest = torch.mean(imbalancetest_epoch).detach()

    infeasibilitytest = torch.mean(infeasibilitytest_epoch).detach()

    relcotest = torch.mean(relcotest_epoch).detach()
    costtest = torch.mean(cost_epoch).detach()
    
    #====== convergence criteria
    infeasible_cases_tot = torch.sum(infeasibility_count).detach()
    line_violation_tot = torch.sum(line_violation_count)
    crit_violation_tot = torch.sum(crit_violation_count)
    #tot_avg_violation = torch.sum(avg_violation_list).detach()
    count_violation_tot = torch.sum(count_violation_list).detach()
    
    error[epoch,:] = imbalancetest.detach().numpy()
    avg_infeasibility[epoch,:] =  infeasibilitytest.detach().numpy()
    relcost[epoch,:] =  relcotest.detach().numpy()
    cost[epoch,:] =  costtest.detach().numpy()
    end_test = time.time()
    time_test = end_test-start_test

    print(epoch, " | ", round(time_test,0), " | ",np.round(error[epoch,:],3), " | ", np.round(torch.mean(clil).detach().numpy(),3), " | ", np.round(avg_infeasibility[epoch,:],3), " | ", np.round(relcost[epoch,:], 3) )
        

print('method time:', time_test)

print('current infeasibility:', (infeasible_cases_tot/Nsamples)*100, '%', 'goal:', 1, '%')
    
#print('current avg violation:', (tot_avg_violation/initial_violation_count),'pu', 'goal', 0.01*max_f, 'pu')
print('infeasible contingency cases:', line_violation_tot, (line_violation_tot/(Ncontingencies*Nsamples))*100, '%')
print('10%+ infeasible cases:', crit_violation_tot, (crit_violation_tot/(Ncontingencies*Nsamples))*100, '%')
print('number of violations:', count_violation_tot)
print('total cost:', cost.sum())

It |   Time   |    Imbalance    | Mod out  |   Infeasibility   |  Rel cost


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.66it/s]

0  |  6.0  |  [168.42]  |  0.478  |  [-0.]  |  [0.554]
method time: 6.014103889465332
current infeasibility: tensor(0.) % goal: 1 %
infeasible contingency cases: tensor(117624.) tensor(2.1559) %
10%+ infeasible cases: tensor(50286.) tensor(0.9217) %
number of violations: tensor(145663.)
total cost: 2155.0712890625


In [ ]:
#============== Create new training data (with high variability from RES)
file_path = 'trained models/training data/Xtest2.pkl' 
with open(file_path, 'rb') as f:
    X = pickle.load(f)
Xtrain = torch.tensor(X[0:int(Nsamples),:],dtype = torch.float32)        
Xtrain_transpose = Xtrain.transpose(0,1)
#Xtest = torch.tensor(X[int(train*Nsamples):,:],dtype = torch.float64)
#standardise
Xmin, Xmax,Xmean,Xstd = np.min(X, axis = 0),np.max(X, axis = 0),np.mean(X, axis = 0),np.std(X, axis = 0)
Xscal = torch.tensor(( X - Xmean ) / Xstd, dtype=torch.float32)

In [ ]:
#=============== Benchmark #2 heuristic approach
if case == 'N-1':
    benchmark = 'SCOPF1screening'

if benchmark == 'SCOPF1screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),1)))
elif benchmark == 'SCOPF2screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),2)))
elif benchmark == 'SCOPF3screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),3)))
elif benchmark == 'SCOPF4screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),4)))

Pgisscopf = torch.zeros(Ngens,int(Nsamples),dtype = torch.float32)
Fl0sscopf = torch.zeros(Nlines,int(Nsamples),dtype = torch.float32)
#Flcs = torch.zeros(Nlines,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
lam1s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
lam2s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
#lam1sc = torch.zeros(Nbus,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
#lam2sc = torch.zeros(Nbus,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
Th0 = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)

time_scopfs = time.time()

Nk_zeros = list(range(len(Nk_contingencies)))
zero_indices = [l for l in Nk_zeros if l not in lcontingencies]
print(lcontingencies)


if benchmark == 'SCOPF1screening':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf1_screening(data, lcontingencies)
elif benchmark == 'SCOPF2screening':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf2_screening(data, lcontingencies)
elif benchmark == 'SCOPF3screening':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf3_screening(data, lcontingencies)
elif benchmark == 'SCOPF4screening':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf4_screening(data, lcontingencies)

cg0scopf.value = np.array(data['gen']['cost'])
for entry in tqdm(range(int(Nsamples)),position=0, leave=True):   
    Pd0scopf.value = X[entry,:] #np.array(data['load']['p'])/Sbase    
    try:
        solution  = problem0scopf.solve(solver=cp.ECOS)    # cp.MOSEK
        if problem0scopf.status in ["infeasible", "unbounded"]:
            # Otherwise, problem.value is inf or -inf, respectively.
            print("Problem infeasible or unbounded")
            #print("Optimal value: %s" % problem0scopf.value)
        Pgisscopf[:,entry] = torch.tensor(Pgiscopf.value)
        Fl0sscopf[:,entry] = torch.tensor(Fl0scopf.value)
        Th0[:,entry] = torch.tensor(Th0scopf.value)

        lam1s[:,entry] = torch.tensor(lam1.value)
        lam2s[:,entry] = torch.tensor(lam2.value)
    except cp.error.SolverError:
        Pgisscopf[:,entry] = Pgisscopf[:,(entry-1)]
        Fl0sscopf[:,entry] = Fl0sscopf[:,(entry-1)]
        Th0[:,entry] = Th0[:,(entry-1)]

        lam1s[:,entry] = lam1s[:,(entry-1)]
        lam2s[:,entry] = lam2s[:,(entry-1)]

        Xtrain[entry,:] = Xtrain[(entry-1),:]

        print("Solver unable to solve for entry:", entry)
        continue

cost_scopf = torch.matmul(gencost,Pgisscopf)

#============ compute Nk flows
Flis = Fl0sscopf

count_crit_violation = 0
count_line_violation = 0

for b in (range(tot_lodf_batches)):

        lodf_batch = lodf_batch_size
        if b == (tot_lodf_batches - 1):
            lodf_batch = last_batch_size
        left_ones = torch.ones((lodf_batch,1))
        right_ones = torch.ones((int(l*batch_size),1))
        Fli_0 = Flis.expand(lodf_batch, -1, -1)

        summed_tensor = torch.zeros(lodf_batch).to(device)

        index1 = LODF_dict[b][0].detach()
        index2 = LODF_dict[b][1].detach()
        indices = torch.stack((index1,index2)).detach()
        values = LODF_dict[b][2].detach()
        shape = (l,int(l*lodf_batch))
        lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

        for k in range(int(Nsamples/batch_size)):
            start = k*batch_size
            end = (k+1)*batch_size
            imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0[:,:,start:end] + torch.sparse.mm(lodf,Flis[:,start:end]).reshape((lodf_batch,l,batch_size)))- max_f)).detach()

            count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > tollerance_crit*max_f, dim=1)
            count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)

            count_crit_violation += torch.count_nonzero(count_crit_violation_loop)
            count_line_violation += torch.count_nonzero(count_line_violation_loop)

crit_violation_tot = count_crit_violation
line_violation_tot = count_line_violation

print('infeasible N-k contingency cases:', line_violation_tot, (line_violation_tot/(Ncontingencies*Nsamples))*100, '%')
print('10%+ contingency cases:', crit_violation_tot, (crit_violation_tot/(Ncontingencies*Nsamples))*100, '%')
    
time_scopfs = time.time() - time_scopfs
print('heuristic approach time:', time_scopfs)

##============= check infeasibility
infeasibility_base = (lam1s + lam2s).permute(1,0)
infeasibility_base[abs(infeasibility_base) < 1e-4] = 0
infeasible_buses_base = (abs(infeasibility_base) > 1e-4) #0.001*load_profile) # 10% of total load available for generation/load shedding
scopf_cost_average = cost_scopf.mean()

infeasible_cases_base = torch.zeros((Nsamples,1))
row_has_true_base = torch.any(infeasible_buses_base, axis=1)
infeasible_cases_base[row_has_true_base] = True
infeasible_cases_tot_base = infeasible_cases_base.sum()
print('infeasible base cases:', infeasible_cases_tot_base)
print('percentage base infeasibility:', (infeasible_cases_tot_base/(Nsamples))*100, '%')
print('total average cost:', scopf_cost_average)

print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

In [ ]:
start_test = time.time()
#============== Initialize training testing data
testing_dataset = Data.TensorDataset(Xscal, cost_scopf)
test_loader = Data.DataLoader(dataset=testing_dataset, batch_size=batch_size, shuffle=False)  # Note: shuffle=False for testing
import torch.nn.functional as F

#================ Initialize tensors
error = np.zeros(shape=(Niterations,1))
avg_infeasibility = np.zeros(shape=(Niterations,1))
relcost = np.zeros(shape=(Niterations,1))
cost = np.zeros(shape=(Niterations,1))

Pgis = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
Flis = torch.zeros(size = (Nlines,int(batch_size)),dtype = torch.float32)
clil = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
infeasibility = torch.zeros(int(batch_size), requires_grad = False)

relcotest_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)
cost_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)

imbalancetest_epoch = torch.zeros(size = (round((Nsamples)/batch_size),1),dtype = torch.float32, requires_grad = False)

infeasibilitytest_epoch = torch.zeros(size = (int(Nsamples/batch_size),Nbus),dtype = torch.float32, requires_grad = False)
infeasibility_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

line_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
crit_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
avg_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
count_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

print("It |   Time   |    Imbalance    | Mod out  |   Infeasibility   |  Rel cost")
for epoch in range(1):    
    with torch.no_grad():
        for i, (Xbatch, cost_scopf_batch) in enumerate(tqdm(test_loader)):
                
                model.eval() # deactivate dropout, when I changed this, testing error stayed same

                Xbatch = Xbatch.clone().detach()
                cost_scopf_batch = cost_scopf_batch.clone().detach()

                Xtest_batch = ((Xbatch * Xstd)+Xmean).to(torch.float32)
            
                #================== Perform forward pass
                #clil = (model(Xbatch))
                clil = (model(Xbatch)+1)/2
                
                Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtest_batch, clil, solver_args=ECOS_solver_args)
                Pgis = Pgi.T
                Flis = Fli.T.to(torch.float32)
                
                #================ compute cost and infeasibility
                infeasibility_batch = sum(lam1i.detach().numpy()) + sum(lam2i.detach().numpy())
                infeasibility_batch = torch.tensor(infeasibility_batch)
                infeasibility_tensor = lam1i + lam2i
                infeasibility_tensor[abs(infeasibility_tensor) < 1e-5 ] = 0
                infeasible_buses = (abs(infeasibility_tensor) > 1e-5)
                
                infeasible_cases = torch.zeros((batch_size,1))
                row_has_true = torch.any(infeasible_buses, axis=1)
                infeasible_cases[row_has_true] = True
                infeasible_cases = infeasible_cases.sum()
                
                batch_cost = torch.matmul(gencost,Pgis)
                batch_relco = ((batch_cost-cost_scopf_batch)/cost_scopf_batch)*100

                #===================== compute contingency flows for each sample
                imbalancetest_batch = 0
                count_crit_violation = 0
                count_line_violation = 0
                count_violation = 0

                for b in (range(tot_lodf_batches)):
                    lodf_batch = lodf_batch_size
                    if b == (tot_lodf_batches - 1): 
                        lodf_batch = last_batch_size
                    left_ones = torch.ones((lodf_batch_size,1))
                    right_ones = torch.ones((int(l*batch_size),1))
                    Fli_0 = Flis.expand(lodf_batch, -1, -1)#.contiguous().view(-1, len(Flis[1])).detach()#.to(torch.float32).clone()
                    #mask = mask_base(b, lodf_batch)
                    #Fli_0[~mask] = 0

                    index1 = LODF_dict[b][0].detach()
                    index2 = LODF_dict[b][1].detach()
                    indices = torch.stack((index1,index2)).detach()
                    values = LODF_dict[b][2].detach()
                    shape = (l,int(l*lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices,values,size = shape).T.to(device)

                    imbalancetest_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf,Flis).reshape((lodf_batch,l,batch_size)))- max_f))
                    imbalancetest_batch_loop[abs(imbalancetest_batch_loop) < 1e-5] = 0
                    
                    count_crit_violation_loop = torch.sum(imbalancetest_batch_loop > tollerance_crit*max_f, dim=1)
                    count_line_violation_loop = torch.sum(imbalancetest_batch_loop > 0, dim=1)
                    imbalancetest_batch_loop = imbalancetest_batch_loop.reshape(lodf_batch,int(l*batch_size))
                    count_violation_loop = torch.count_nonzero(imbalancetest_batch_loop)
                
                    imbalancetest_batch += torch.sum(imbalancetest_batch_loop)/int(batch_size)
                
                    count_violation += count_violation_loop
                    count_crit_violation += torch.count_nonzero(count_crit_violation_loop.detach())
                    count_line_violation += torch.count_nonzero(count_line_violation_loop.detach())
        
                relcotest_epoch[i,:] = batch_relco.detach()
                cost_epoch[i,:] = batch_cost.detach()
                imbalancetest_epoch[i,:] = imbalancetest_batch.detach()
                infeasibilitytest_epoch[i,:] = infeasibility_batch.detach()
                
                infeasibility_count[i,:] = infeasible_cases.detach()
                line_violation_count[i,:] = count_line_violation.detach()
                crit_violation_count[i,:] = count_crit_violation.detach()
                #avg_violation_list[i,:] = avg_violation.detach()
                count_violation_list[i,:] = count_violation
             

    imbalancetest = torch.mean(imbalancetest_epoch).detach()

    infeasibilitytest = torch.mean(infeasibilitytest_epoch).detach()

    relcotest = torch.mean(relcotest_epoch).detach()
    costtest = torch.mean(cost_epoch).detach()
    
    #====== convergence criteria
    infeasible_cases_tot = torch.sum(infeasibility_count).detach()
    line_violation_tot = torch.sum(line_violation_count)
    crit_violation_tot = torch.sum(crit_violation_count)
    #tot_avg_violation = torch.sum(avg_violation_list).detach()
    count_violation_tot = torch.sum(count_violation_list).detach()
    
    error[epoch,:] = imbalancetest.detach().numpy()
    avg_infeasibility[epoch,:] =  infeasibilitytest.detach().numpy()
    relcost[epoch,:] =  relcotest.detach().numpy()
    cost[epoch,:] =  costtest.detach().numpy()
    end_test = time.time()
    time_test = end_test-start_test

    print(epoch, " | ", round(time_test,0), " | ",np.round(error[epoch,:],3), " | ", np.round(torch.mean(clil).detach().numpy(),3), " | ", np.round(avg_infeasibility[epoch,:],3), " | ", np.round(relcost[epoch,:], 3) )
        

print('method time:', time_test)

print('current infeasibility:', (infeasible_cases_tot/Nsamples)*100, '%', 'goal:', 1, '%')
    
#print('current avg violation:', (tot_avg_violation/initial_violation_count),'pu', 'goal', 0.01*max_f, 'pu')
print('infeasible contingency cases:', line_violation_tot, (line_violation_tot/(Ncontingencies*Nsamples))*100, '%')
print('10%+ infeasible cases:', crit_violation_tot, (crit_violation_tot/(Ncontingencies*Nsamples))*100, '%')
print('number of violations:', count_violation_tot)
print('total cost:', cost.sum())